In [1]:
import cv2
import pandas as pd
import re

In [38]:
geo_df = pd.read_csv('./geo_all.csv', names=['タイトル', '監督', 'カテゴリ', 'キャスト', '販売国', '画像URL', '詳細URL'])

In [39]:
# カテゴリの重複を削除してユニークにする
geo_categories_column = geo_df['カテゴリ'].apply(lambda x: str(x).split(','))

In [40]:
geo_categories_column[0], geo_categories_column[0][0]

(['海外TVドラマ', 'ミステリー・サスペンス'], '海外TVドラマ')

In [41]:
from itertools import chain

In [42]:
geo_categories = list(set(chain.from_iterable(geo_categories_column.values.flatten().tolist())))

In [43]:
geo_categories, len(geo_categories)

(['ゲーム',
  'モータースポーツ',
  '戦争',
  'アドベンチャー',
  'クラシック',
  'エロス',
  '名作（アニメ）',
  '青春',
  'SF',
  '伝記',
  '戦争ドラマ',
  '国内スポーツ',
  'その他',
  '邦画',
  '子供が主役',
  '競馬',
  '水着',
  'オムニバス',
  'コメディ',
  'ファンタジー',
  'ギャング・マフィア',
  '青春 （アニメ）',
  'アイドル',
  '海外音楽',
  'ハードボイルド',
  '3D',
  '海外TVドラマ',
  '自然・ヒーリング',
  '国内TVドラマ',
  'スノーボード・スキー',
  'プロレス',
  'メイキング',
  '美少女（アニメ）',
  '動物',
  'ディズニー',
  'キッズ',
  'ファミリー',
  '漫才・コント',
  '鉄道・旅',
  'ファミリー （アニメ）',
  'パニック',
  '国内音楽',
  '海外スポーツ',
  'SF （アニメ）',
  'SM',
  '国内TVバラエティ',
  'ファンタジー（アニメ）',
  'ミステリー・サスペンス',
  '落語',
  'HOW TO',
  '時代劇',
  '演劇・ミュージカル',
  'パチンコ＆パチスロ',
  'カンフー・少林寺',
  'ホラー',
  '教養・ドキュメント',
  '国内アニメ',
  '音楽',
  '法廷ドラマ',
  'ウエスタン',
  'ヒーロー （アニメ）',
  '文芸・史劇',
  'オリジナル （アニメ）',
  '冒険（アニメ）',
  'ボーイズラブ （アニメ）',
  '刑事',
  'スパイ',
  'ドラマ',
  '海外アニメ',
  '洋画',
  'アクション',
  'コミック（アニメ）',
  'アクション（アニメ）',
  '格闘技',
  'サッカー',
  '海外TVバラエティ',
  'ジブリ',
  'スプラッター',
  'お笑い・バラエティ',
  'ヤクザ',
  'キッズ （アニメ）',
  'ヒーロー',
  '企画・ドキュメント',
  'スポーツ',
  'アジア',
  'バイオレンス',
  'ラブロマ

この中の、（個人的に好きなジャンル）'ホラー', 'エイリアン・モンスター', 'パニック', 'SF'、 'ミステリー・サスペンス'を予測するモデルを作成する

In [44]:
my_favorite = ['ホラー', 'エイリアン・モンスター', 'パニック', 'SF', 'ミステリー・サスペンス']
# アニメ、ドラマシリーズは取り除きたい
ignore_categories = ['国内TVドラマ', 'アジアTVドラマ', '国内アニメ', '海外TVドラマ', 'SF （アニメ）']

In [45]:
_my_favarit_df = geo_df[geo_df['カテゴリ'].str.contains('|'.join(my_favorite))]
my_favarit_df = _my_favarit_df[~_my_favarit_df['カテゴリ'].str.contains('|'.join(ignore_categories))]

In [33]:
my_favarit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11055 entries, 12 to 133048
Data columns (total 7 columns):
タイトル     11055 non-null object
監督       10557 non-null object
カテゴリ     11055 non-null object
キャスト     9972 non-null object
販売国      11055 non-null object
画像URL    11055 non-null object
詳細URL    11055 non-null object
dtypes: object(7)
memory usage: 690.9+ KB


とりあえず、11,055は集まったっぽい

In [34]:
# csvにはいてラベル付する
my_favarit_df.to_csv('favarit.csv')

In [36]:
# と、思ったけど、ラベル付だるくなったので、好きなやつだけにする
# つまり、好きっぽい・あんま興味ない範囲外の映画にする

In [57]:
not_my_favarit_df = geo_df[~geo_df.index.isin(my_favarit_df.index)]

In [58]:
not_my_favarit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122011 entries, 0 to 133065
Data columns (total 7 columns):
タイトル     122011 non-null object
監督       118023 non-null object
カテゴリ     122011 non-null object
キャスト     106675 non-null object
販売国      122011 non-null object
画像URL    122011 non-null object
詳細URL    122011 non-null object
dtypes: object(7)
memory usage: 7.4+ MB


これでいいっぽい

In [71]:
my_favarit_df.loc[:, 'label'] = 0
not_my_favarit_df.loc[:, ('label')] = 1

/Users/kazumatamaki/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [72]:
my_favarit_df.head()

,タイトル,監督,カテゴリ,キャスト,販売国,画像URL,詳細URL,label
12,ほんとにあった！呪いのビデオ 34,児玉和土,"その他,教養・ドキュメント,ホラー",中村義洋,日本,https://rental.geo-online.co.jp/cdn/00563000/0...,http://rental.geo-online.co.jp//detail-147378....,0
29,【Blu-ray】アンダーワールド ビギンズ,パトリック・タトポロス,"洋画,アクション,SF","マイケル・シーン,ビル・ナイ,ローナ・ミトラ,スティーヴン・マッキントッシュ",アメリカ,https://rental.geo-online.co.jp/cdn/00561000/0...,http://rental.geo-online.co.jp//detail-149229....,0
50,REC/レック 2,ジャウマ・バラゲロ,"洋画,ホラー,パニック","パコ・プラサ,ジョナサン・メイヨール,オスカル・サンチェス・サフラ,マニュエラ・ヴェラスコ",スペイン,https://rental.geo-online.co.jp/cdn/00593000/0...,http://rental.geo-online.co.jp//detail-176425....,0
51,悪夢のエレベーター,堀部圭亮,"邦画,ミステリー・サスペンス,コメディ","内野聖陽,佐津川愛美,モト冬樹,斉藤工",日本,https://rental.geo-online.co.jp/cdn/00593000/0...,http://rental.geo-online.co.jp//detail-176427....,0
55,ラスト3デイズ 〜すべて彼女のために〜,フレッド・カヴァイエ,"洋画,ミステリー・サスペンス","ダイアン・クルーガー,ヴァンサン・ランドン,ランスロ・ロッシュ,オリヴィエ・マルシャル",フランス,https://rental.geo-online.co.jp/cdn/00624000/0...,http://rental.geo-online.co.jp//detail-191651....,0


In [73]:
not_my_favarit_df.head()

,タイトル,監督,カテゴリ,キャスト,販売国,画像URL,詳細URL,label
0,LOST シーズン5 Vol.5,マシュー・フォックス,"海外TVドラマ,ミステリー・サスペンス","エヴァンジェリン・リリー,ジョシュ・ホロウェイ,ナヴィーン・アンドリュース,エミリー・デ・レイビン",アメリカ,https://rental.geo-online.co.jp/cdn/00583000/0...,http://rental.geo-online.co.jp//detail-170348....,1
1,LOST シーズン5 Vol.6,マシュー・フォックス,"海外TVドラマ,ミステリー・サスペンス","エヴァンジェリン・リリー,ジョシュ・ホロウェイ,ナヴィーン・アンドリュース,エミリー・デ・レイビン",アメリカ,https://rental.geo-online.co.jp/cdn/00583000/0...,http://rental.geo-online.co.jp//detail-170349....,1
2,NHK大河ドラマ 天地人 完全版 第4巻,片岡敬司,"国内TVドラマ,時代劇","妻夫木聡,北村一輝,小栗旬,阿部寛",日本,https://rental.geo-online.co.jp/cdn/00580000/0...,http://rental.geo-online.co.jp//detail-168838....,1
3,異世界の聖機師物語 3,吉川浩司,"国内アニメ,ファンタジー（アニメ）","下野紘,米澤円,桑谷夏子,恒松あゆみ",日本,https://rental.geo-online.co.jp/cdn/00557000/0...,http://rental.geo-online.co.jp//detail-145841....,1
4,NHKドラマスペシャル 白洲次郎 1,大友啓史,国内TVドラマ,"伊勢谷友介,中谷美紀,奥田瑛二,原田美枝子",日本,https://rental.geo-online.co.jp/cdn/00573000/0...,http://rental.geo-online.co.jp//detail-165716....,1


In [74]:
my_favarit_df.to_csv('favarit.csv')
not_my_favarit_df.to_csv('not_my_favarit.csv')

In [2]:
test = pd.read_csv('./test.csv')

In [7]:
test[['画像URL']].to_csv('./test.csv', header=False, index=True)